In [ ]:
import os
import subprocess
from varseek.utils import create_mutant_t2g

ref_fa = "/home/jrich/Desktop/varseek/tests/ref_test.fa"
read1_fq = "/home/jrich/Desktop/varseek/tests/reads1_test.fq"
read2_fq = "/home/jrich/Desktop/varseek/tests/reads2_test.fq"
test_index = "/home/jrich/Desktop/varseek/tests/index_test.idx"
test_t2g = "/home/jrich/Desktop/varseek/tests/t2g_test.txt"
kb_count_out_test = "/home/jrich/Desktop/varseek/tests/kb_files/test_kb_count_out"

rnaseq_fastq_files_final = [read1_fq, read2_fq]
os.makedirs(kb_count_out_test, exist_ok=True)

if not os.path.exists(test_index):
    kb_ref_command = ["kb", "ref", "--workflow", "custom", "-t", str(8), "-i", test_index, "--d-list", "None", ref_fa]
    subprocess.run(kb_ref_command, check=True)

if not os.path.exists(test_t2g):
    create_mutant_t2g(ref_fa, test_t2g)

if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
    kb_count_command = ["kb", "count", "-t", str(8), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

# kb_count_out_test = "/home/jrich/Desktop/varseek/tests/kb_files/test_kb_count_out_paired"
# if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(8), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

In [ ]:
bustools = "/home/jrich/miniconda3/envs/varseek2/lib/python3.10/site-packages/kb_python/bins/linux/bustools/bustools"
bus_text_file = os.path.join(kb_count_out_test, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

In [ ]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.seq_utils)
from varseek.utils.seq_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, assay = "bulk", bustools = bustools)